In [186]:
import pandas as pd
import numpy as np
import time
import datetime
import tensorflow as tf

In [55]:
properties_read = pd.read_csv('properties_2017.csv')
train_read = pd.read_csv('train_2017.csv')
print(properties.shape)
print(train.shape)

/Users/arthurhlaing/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2985217, 58)
(77613, 81)


In [187]:
# Add features to train
train = train_read.merge(properties_read, left_on='parcelid', right_on='parcelid', how='left')

In [188]:
# Finds the number of missing values in each column
num_of_na = [train.loc[:,col].isnull().sum() for col in train]
# Divide by rows for proportion 
prop_na = [num / train.shape[0] for num in num_of_na]
# Put the proporitons and column names into a df
na_df = pd.DataFrame({'prop_na' : prop_na, 'column' : train.columns})
na_df = na_df.sort_values('prop_na')
print(na_df)

     prop_na                        column
0   0.000000                      parcelid
1   0.000000                      logerror
2   0.000000               transactiondate
27  0.000438                     longitude
34  0.000438     propertycountylandusecode
35  0.000438         propertylandusetypeid
37  0.000438        rawcensustractandblock
39  0.000438                regionidcounty
42  0.000438                       roomcnt
19  0.000438                          fips
7   0.000438                    bedroomcnt
6   0.000438                   bathroomcnt
54  0.000438                assessmentyear
26  0.000438                      latitude
53  0.000451             taxvaluedollarcnt
55  0.000464         landtaxvaluedollarcnt
56  0.000502                     taxamount
41  0.001082                   regionidzip
52  0.001920    structuretaxvaluedollarcnt
13  0.003028  calculatedfinishedsquarefeet
59  0.003621           censustractandblock
49  0.003917                     yearbuilt
21  0.00837

In [190]:
threshold = 0.60
train_filtered = train.loc[:, (train.isnull().sum(axis=0) <= (train.shape[0]*threshold))]
print(train.shape)

(77613, 60)


In [191]:
num_of_na = [train_filtered.loc[:,col].isnull().sum() for col in train_filtered]
# Divide by rows for proportion 
prop_na = [num / train_filtered.shape[0] for num in num_of_na]
# Put the proporitons and column names into a df
na_df = pd.DataFrame({'prop_na' : prop_na, 'column' : train_filtered.columns})
na_df = na_df.sort_values('prop_na')
print(na_df)

     prop_na                        column
0   0.000000                      parcelid
1   0.000000                      logerror
2   0.000000               transactiondate
27  0.000438                assessmentyear
22  0.000438                       roomcnt
20  0.000438                regionidcounty
18  0.000438        rawcensustractandblock
16  0.000438         propertylandusetypeid
13  0.000438                     longitude
12  0.000438                      latitude
15  0.000438     propertycountylandusecode
9   0.000438                          fips
4   0.000438                    bedroomcnt
3   0.000438                   bathroomcnt
26  0.000451             taxvaluedollarcnt
28  0.000464         landtaxvaluedollarcnt
29  0.000502                     taxamount
21  0.001082                   regionidzip
25  0.001920    structuretaxvaluedollarcnt
7   0.003028  calculatedfinishedsquarefeet
30  0.003621           censustractandblock
24  0.003917                     yearbuilt
6   0.00837

In [194]:
# Replace Na with 0
train_filtered = train_filtered.fillna(0)
date_strings = (train_filtered.values[:,2])
date_timestamps = []
for string in date_strings:
    date_timestamps.append(time.mktime(datetime.datetime.strptime(string, "%Y-%m-%d").timetuple()))
train_filtered['transactiondate'] = np.asarray(date_timestamps)

In [195]:
train_range = train_filtered.max() - train_filtered.min()
train_mean = train_filtered.mean()

In [196]:
#identify numeric columns to normalize
numeric_cols = train_filtered.columns.drop(['propertycountylandusecode', 'propertyzoningdesc'])
train_filtered = train_filtered.drop(columns=['propertycountylandusecode', 'propertyzoningdesc'])

In [197]:
for col in numeric_cols:
    train_filtered[col] = (train_filtered[col]-train_mean[col])/train_range[col]

In [181]:
train_filtered

,parcelid,logerror,transactiondate,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,...,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
0,0.008216,0.000891,-0.516983,0.066806,0.059257,-0.349427,0.066834,0.037050,0.064910,0.00210,...,-0.241402,-0.105402,-0.003073,-0.973152,-0.01654,-0.009986,-1.000058,-0.006149,-0.010216,-0.124791
1,0.025769,0.003918,-0.516983,-0.072083,-0.065743,-0.349427,-0.072055,-0.008825,-0.009649,0.01061,...,-0.241402,-0.105395,-0.003073,-0.973152,-0.01654,-0.009986,-1.000058,-0.006149,-0.010216,-0.124791
2,0.007507,-0.001147,-0.516983,-0.016527,-0.003243,-0.349427,-0.016500,-0.015054,-0.019773,0.00210,...,-0.241402,-0.105394,-0.003073,-0.973152,-0.01654,-0.009986,-1.000058,-0.006149,-0.010216,-0.124791
3,-0.005287,-0.012115,-0.516983,0.039028,0.059257,0.317240,0.039056,0.016736,0.031894,-0.00150,...,-0.241402,-0.105402,-0.003073,-0.973152,-0.01654,-0.009986,-1.000058,-0.006149,-0.010216,-0.124791
4,-0.013509,-0.000990,-0.516983,0.039028,-0.003243,0.317240,0.039056,-0.013118,-0.016626,-0.00150,...,-0.241402,-0.105402,-0.003073,-0.973152,-0.01654,-0.009986,-1.000058,-0.006149,-0.010216,-0.124791
5,0.026344,-0.003759,-0.516983,-0.016527,-0.003243,-0.349427,-0.016500,-0.008068,-0.008418,0.01061,...,-0.241402,-0.105394,-0.003073,-0.973152,-0.01654,-0.009986,-1.000058,-0.006149,-0.010216,-0.124791
6,-0.005814,-0.001791,-0.516983,0.039028,0.059257,0.400573,0.039056,0.033178,0.058617,-0.00150,...,-0.241402,-0.105402,-0.003073,-0.973152,-0.01654,-0.009986,-1.000058,-0.006149,-0.010216,-0.124791
7,-0.005980,0.008567,-0.516983,-0.072083,-0.065743,0.067240,-0.072055,-0.029224,-0.042802,-0.00150,...,-0.241402,-0.105402,-0.003073,-0.973152,-0.01654,-0.009986,-1.000058,-0.006149,-0.010216,-0.124791
8,-0.001384,-0.005820,-0.513237,0.039028,0.059257,0.400573,0.039056,0.035339,0.062128,-0.00150,...,-0.241402,-0.105402,-0.003073,-0.973152,-0.01654,-0.009986,-1.000058,-0.006149,-0.010216,-0.124791
9,-0.009334,-0.005396,-0.513237,0.039028,-0.065743,0.317240,0.039056,-0.013735,-0.017629,-0.00150,...,-0.241402,-0.105402,-0.003073,-0.973152,-0.01654,-0.009986,-1.000058,-0.006149,-0.010216,-0.124791


In [198]:
train_y = train_filtered.values[:,1]
train_y = train_y.reshape(train_y.shape[0],1)
train_x = train_filtered.values[:,2:]
print(train_x.shape)
print(train_y.shape)

(77613, 27)
(77613, 1)


In [201]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(3, input_shape=(27,), activation=tf.nn.relu),
        tf.keras.layers.Dense(3, activation=tf.nn.relu),
        tf.keras.layers.Dense(1)
    ])
sgd = tf.keras.optimizers.SGD(lr=0.1)
model.compile(optimizer=sgd, loss='mean_squared_error', metrics=['t'])

In [203]:
model.fit(train_x, train_y, verbose=1, epochs = 10)

Epoch 1/10
77613/77613 [==============================] - 5s 58us/step - loss: 2.9794e-04 - acc: 0.0000e+00
Epoch 2/10
77613/77613 [==============================] - 4s 58us/step - loss: 2.9790e-04 - acc: 0.0000e+00
Epoch 3/10
77613/77613 [==============================] - 5s 62us/step - loss: 2.9788e-04 - acc: 0.0000e+00: 4s - loss: 2.6732e-04 - ETA: 0s - loss: 2.9418e-04 -
Epoch 4/10
77613/77613 [==============================] - 5s 64us/step - loss: 2.9781e-04 - acc: 0.0000e+00: 1
Epoch 5/10
77613/77613 [==============================] - 5s 67us/step - loss: 2.9793e-04 - acc: 0.0000e+00
Epoch 6/10
77613/77613 [==============================] - 5s 63us/step - loss: 2.9772e-04 - acc: 0.0000e+00
Epoch 7/10
77613/77613 [==============================] - 5s 62us/step - loss: 2.9785e-04 - acc: 0.0000e+00
Epoch 8/10
77613/77613 [==============================] - 5s 65us/step - loss: 2.9783e-04 - acc: 0.0000e+00
Epoch 9/10
77613/77613 [==============================] - 6s 73us/step - loss: 

In [205]:
model.evaluate(train_x, train_y, verbose = 1)

77613/77613 [==============================] - 3s 36us/step


[0.000296911573062204, 0.0]